In [86]:
import numpy as np
import pandas as pd 
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [87]:
# loading movie data from csv file to a pandas dataframe
movies_data = pd.read_csv(r"C:\Users\devin\OneDrive\Desktop\MRWP\movies.csv")

In [88]:
# printing the first 5 rows of the dataframe (sampling the data)
# we do so to gain an understanding of the dataset we are working with
movies_data.head()

# I tried using the sample method but the data doesn't look as clean as simply calling head()
# sample_data = movies_data.sample(frac=0.1)
# print(sample_data)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [89]:
# number of rows and columsn in the movie data frame
movies_data.shape

# Check for missing values
missing_values = movies_data.isnull().sum()
print(missing_values)

# Remove duplicates
# Not sure if this is necessary yet
# I'll wait until I finish the youtube course to try this
# movies_data = movies_data.drop.duplicates() 

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64


In [90]:
# selecting the relevant features for recommendation (feature selection)
# content/popularity based recommendation system
# Future optiimization: Consider including the movie's plot in the selected features as well
#                       Maybe allow the user to choose to enable plotline to get movies more similar to the one entered
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [91]:
# replacing the null (missing) selected feature values with null (empty) string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [92]:
# combining all the 5 selected features
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

# if features need to be combined more than 3 times, use this function
#def combine_features(row):
#    return row['genres'] + ' ' + row['keywords'] + ' ' + row['tagline'] + ' ' + row['cast'] + ' ' + row['director']

#combined_features = movies_data.apply(combine_features, axis=1)

In [93]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [94]:
# convert text data to feature vectors (numberical values)

#Load the vectorizer function into the variable
vectorizer = TfidfVectorizer()

In [95]:
# Transform all the combined features data into numerical values and save those values
feature_vectors = vectorizer.fit_transform(combined_features)

In [96]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Cosine Similarity

In [97]:
# calculating similarity scores using consine similarity
# getting a numerical score value for all movies

similarity = cosine_similarity(feature_vectors)

In [98]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [99]:
print(similarity.shape)

(4803, 4803)


Get Movie Name from User

In [100]:
# getting the movie name from user

movie_name = input(' Enter your favourite movie name: ')

In [101]:
# creating a list with all the movie names given in the dataset
# we need this list of movies to compare with the movie name the user inputed
# the system cannot given recommendations for a movie not in the dataset
movie_title_list = movies_data['title'].tolist()
print(movie_title_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [102]:
# find the closest match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, movie_title_list)
print(find_close_match)

['Blade', 'Blade II', 'Made']


In [103]:
# get close matches function may output more than one movie title
# get and save the value of first item title in the list to get the closest match
close_match = find_close_match[0]
print(close_match)

Blade


In [104]:
# finding the index of the movie using the title (closest match)

close_match_index = movies_data[movies_data.title == close_match]['index'].values[0]
print(close_match_index)

1056


In [105]:
# getting a list of similar movies based on the index value and similarity score

similarity_score = list(enumerate(similarity[close_match_index]))
print(similarity_score)

[(0, 0.045674637975104015), (1, 0.0047283834330500716), (2, 0.005542770275163421), (3, 0.0060489974986066355), (4, 0.0052752528918167015), (5, 0.005612080053577803), (6, 0.0), (7, 0.005468238427168521), (8, 0.0), (9, 0.005342471043235664), (10, 0.0060934122733415615), (11, 0.004812599397074285), (12, 0.0055315186944138684), (13, 0.004994192945314782), (14, 0.02182531257058305), (15, 0.0), (16, 0.005185052412103569), (17, 0.005291539791609293), (18, 0.005175969279750806), (19, 0.00469802878444452), (20, 0.005394990231469967), (21, 0.0045465900414059755), (22, 0.01666557792934236), (23, 0.0), (24, 0.005110868626720241), (25, 0.0), (26, 0.005643697609360442), (27, 0.004935050305210084), (28, 0.005418102851314418), (29, 0.00566814234393254), (30, 0.07109363790597789), (31, 0.005324011320540028), (32, 0.0), (33, 0.005434262622494418), (34, 0.0), (35, 0.005704616151767433), (36, 0.005140314391453203), (37, 0.0), (38, 0.005528932841765454), (39, 0.005101978079452634), (40, 0.0), (41, 0.005218

In [106]:
len(similarity_score)

4803

In [107]:
# sort movies based on their similarity scores

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True)
print(sorted_similar_movies)

[(1056, 1.0000000000000002), (654, 0.3013995690119305), (864, 0.2245610026938736), (505, 0.22252331152839894), (4274, 0.201340418025455), (768, 0.19286962758080017), (597, 0.17233244122797747), (2684, 0.14623970603187453), (612, 0.1311557366518496), (3210, 0.12959873291917917), (1939, 0.128882210317194), (1167, 0.12772194164895417), (1265, 0.12668347024562754), (2029, 0.1206336209143486), (1722, 0.11935757006507222), (624, 0.11871287247007797), (589, 0.11731295371017733), (3020, 0.11536615196701033), (2208, 0.11316162025362109), (217, 0.11226727533671833), (3028, 0.1117026726235941), (4450, 0.10806816683644532), (4619, 0.10756497940376517), (1203, 0.1060454624308842), (744, 0.10604094505321678), (3289, 0.10566411049398537), (3387, 0.10327513734093177), (2205, 0.10263563663773582), (1346, 0.10003427519352394), (2044, 0.09991000474849607), (1192, 0.09917704931268363), (2689, 0.0991703548085222), (3225, 0.09889728273824205), (4531, 0.09813453737626196), (3756, 0.09807142243468611), (247, 

In [108]:
# print the list of similar movies based on the index

print('Movies suggested for you: \n')
i = 1
for movie in sorted_similar_movies[1:]:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if (i < 30):
        print(i, '.', title_from_index)
        i += 1

Movies suggested for you: 

1 . Blade: Trinity
2 . Blade II
3 . The League of Extraordinary Gentlemen
4 . Eddie: The Sleepwalking Cannibal
5 . Priest
6 . Underworld: Awakening
7 . Vampire in Brooklyn
8 . The Twilight Saga: Eclipse
9 . The Lost Boys
10 . BloodRayne
11 . Dracula
12 . FearDotCom
13 . Underworld
14 . Dracula 2000
15 . Abraham Lincoln: Vampire Hunter
16 . Dracula Untold
17 . Sugar Hill
18 . Let Me In
19 . Ghost Rider
20 . Mo' Better Blues
21 . American Hero
22 . First Love, Last Rites
23 . Secret Window
24 . The Lego Movie
25 . The Wendell Baker Story
26 . Somewhere
27 . Undisputed
28 . Reindeer Games
29 . The Little Vampire
